In [ ]:
%matplotlib widget
from IPython.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))

# The Optimizer Framework and UI Functions

The `Optimizer` framework allows rapid implementation of notebooks for Proof of Concepts (PoC) and the demonstration and improvement of Digital Annealer models. The traveling salesman problem as combinatorial optimization example is taken here to present step by step how to realize not only the QUBO but a test and optimization frame to execute and tune the whole process. For the tutorial please go through the chapters and after each text block run the respective code box. For better understanding you can also try some modifications in the code and see the effects. _And now enjoy the experience!_

## The description
Normally such notebooks start with an explanation of the problem and the mathematical solution. This tutorial uses the traveling salesman problem (TSP) as example content.

QUBO formulations for TSP are well known and besides many others it can be found in [Lucas, Andrew. (2014). Ising formulations of many NP problems. Frontiers in Physics. 2. 5. 10.3389/fphy.2014.00005.](https://www.frontiersin.org/articles/10.3389/fphy.2014.00005/full). 

For $N$ cities $c=0,1,...N-1$ and $N$ time steps $t=0,1,...N-1$ let the bit $x_{t,c}$ express the presence at that time in the city. The QUBO we have to build consists of two parts to enforce constraints, i.e. being at exactly one place in each time step:
$$
P_{time} = f_{rules} * \sum_{t=0}^{N - 1} \left(1 - \sum_{c=0}^{N - 1} x_{t,c} \right)^2
$$
and visiting each city exactly once:
$$
P_{city} = f_{rules} * \sum_{c=0}^{N - 1} \left(1 - \sum_{t=0}^{N - 1} x_{t,c} \right)^2
$$
As third part we have the minimization target, which is the length of the selected travel route:
$$
P_{distance} = f_{distance} *  \sum_{t=0}^{N - 1} \sum_{c_0=0 \\c_1=0 \\c_0 \ne c_1}^{N - 1} \Delta_{c_0, c_1} x_{t,c_0} x_{(t+1) mod N, c_1} 
$$
All three together give our QUBO:
$$
    HQ = P_{time} + P_{city} + P_{distance}
$$

## The Implementation
This chapter shows the implementation of the solution based on the **Optimizer** framework. The `Optimizer` class is part of the `dadk` library and is responsible for the complete control flow and the GUI. The specialities of a concrete solution have to be realized as a class inherited from `OptimizerModel`. The following sections introduce the implementation of the class `Demo_UIFunctions_Model`. It will be done here step by step as a sequence of enriched functionalites of classes `Demo_UIFunctions_Model_0`, `Demo_UIFunctions_Model_1`, etc. The first step just uses the default class without any specific content:
```python
class Demo_UIFunctions_Model_0(OptimizerModel):
    ... (later class definition goes here
    
my_optimizer_0 = Optimizer(Demo_UIFunctions_Model_0())    
```
The `Optimizer` class is called with an instance of `Demo_UIFunctions_Model_0` to open the GUI. Execute the following box to import the module from the `dadk` library and run a first example.

In [ ]:
from dadk.Optimizer import *

class Demo_UIFunctions_Model_0(OptimizerModel):
    def __init__(self, persistent_file="O_02_Optimizer_UI_functions.dao", **kwargs):
        OptimizerModel.__init__(
            self,
            name = 'O_02_Optimizer_UIFunctions',
            persistent_file=persistent_file,
            **kwargs)

my_optimizer_0 = Optimizer(Demo_UIFunctions_Model_0())    

### The GUI decalaration: `__init__`
The main task of the `__init__` method is the declaration of the GUI. This is done by a call to the super class. The values of the parameters define the GUI. All parameters are optional.

#### The GUI  for **Setup scenario** and **Build QUBO**: `load_parameter` and `build_gui_parameter`
The GUI for the tabs **Setup scenario** and **Build QUBO** have to be defined by a `list` given as value to the respective parameter. The values entered into the GUI are then propagated to the methods `load` and `build_qubo`. Their parameter list is represented as a list consisting of dictionaries:
``` python
load_parameter = [{<parameter 1>}, {<parameter 1>}, ...]
```

defining the GUI elements, which are displayed in the given order. The declaration of a single GUI element looks like this:

``` python
{
    'name':'my_parameter', 
    'value':12, 
    'type':'int_slider', 
    'min':2, 'max': 90, 
     'label':'Size', 
    'description':'problem size', 
    'tags':['demo1', 'demo2']
}
```

The `name` key refers to the name of a parameter of the respective method. `value` is an initial default value; it's type has to be consistent with the parameter of the function and the selected widget type. ``type`` is a
widget type to be used and has one of the following string values: 

``text``, ``text_area``, ``float_bounded``,
``int_bounded``, ``float_slider``, ``float_log_slider``, ``float_range_slider``, ``int_slider``,
``int_range_slider``, ``select``, ``select_slider``, ``select_range_slider``, ``radio``, ``select_multiple``,
``check``, ``checkbox``, ``directory_selector``, ``file_selector``, ``date_picker``, ``color_picker``, ``begin_group``, ``end_group``. 

Depending on the type there are additional attributes. ``min`` and ``max`` define limits of bounded elements and sliders and are numbers or tuples of numbers in case of range sliders. ``step`` defines the step size for sliders. Selection widgets use ``options``, which are given as a list of possible values or tuples of GUI representation and value. 

The types ``begin_group`` and ``end_group`` allow the structuring of the widget list by accordions. The ``begin_group``
starts an accordion and a new group and the ``end_group`` ends the last group and the complete accordion.
Dictionaries with type ``begin_group`` and ``end_group`` must not contain a ``name`` key.

The ``value`` entry defines the initial value and it's type has to be consistent with the parameter of the
function and the selected widget type. Related to the specified ``type`` the entries for the keys ``min``,
``max``, ``step`` or ``options`` define the widget behavior. 

``label`` and ``description`` are used in the GUI to mark the input element for the parameter. 

Under ``tags`` one can specify a list of tags to display or suppress the widget. If the Optimizer is called with ``mode`` parameter then only widgets which are tagged with this mode are displayed. This allows restricted GUIs e.g. for demonstrations.

In [ ]:
class Demo_UIFunctions_Model_1(Demo_UIFunctions_Model_0):
    def __init__(self, persistent_file="O_02_Optimizer_UI_functions.dao", **kwargs):
        Demo_UIFunctions_Model_0.__init__(
            self,
            persistent_file=persistent_file,
            # Parameters and GUI for the "Setup scenario" tab
            load_parameter=[
                {'name':'N', 'type':'int_slider', 'value':10, 'min':2, 'max': 20, 
                 'label':'Size', 'description':'problem size', 'tags':['demo1', 'demo2']},
                {'name':'fixed_city', 'type':'int_slider', 'value':0, 'min':0, 'max':19, 
                 'label':'fixed city', 'description': '', 'tags':['demo1']},
                {'name':'fixed_time', 'type':'int_slider', 'value':0, 'min':0, 'max':19, 
                 'label':'fixed time', 'description':'', 'tags':['demo1']}
            ],
            # Parameters and GUI for the "Build QUBO" tab
            build_qubo_parameter=[
                {'name':'factor_rules', 'type':'float_slider', 'value':100.0, 'min':0.0, 'max':10**6,
                 'label':'factor rules'},
                {'name':'factor_distance', 'type':'float_slider', 'value':1.0, 'min':0.0, 'max':10**3,
                 'label':'factor distance'}
            ],
            **kwargs)

display(HTML("<h3>vanilla-version</h3>"))
my_optimizer_1 = Optimizer(Demo_UIFunctions_Model_1())

display(HTML("<h3>'demo1'-version</h3>"))
my_optimizer_1_demo1 = Optimizer(Demo_UIFunctions_Model_1(), mode='demo1')

display(HTML("<h3>'demo2'-version</h3>"))
my_optimizer_1_demo2 = Optimizer(Demo_UIFunctions_Model_1(), mode='demo2')

### Creation of business data model: `load`:
The tab **Setup scenario** is served by the method `load`. This method realizes the first process step in the Optimizer framework. It sets up the scenario as some business data model. Input data can be read from file or retrieved from any other data source reachable from the used Python Jupyter Notebook platform. Another option is generated data for a first concept model. That is done here. After storing the parameter `N` for the number of cities in an object attribute `N` positions on a circle in the plane are created and the distances between each two cities are calculated.

The parameters of the `load` method can be defined as needed to control the setup. The GUI for the parameters has to be defined consistently as `load_parameter` in the [`__init__` method](#The-GUI-decalaration:-__init__). The method can comment any progress or results by logging:
``` python
        self.LOGGER.info(f'Model for circular map with {self.N} cities created.')
```
The logging levels debug, info, warning, error and critical can be used and the level for output in the tab is set as parameter `logging` when instantiating the Optimizer (see [Variations of the GUI](#Variations-of-the-GUI)).

`print` can be used for more output in a structured form. Since the method is also used in pareto and time-series tests it might make sense to suppress some output there. This can be decided by the value of the parameter `silent` which is `False` for a direct call from the GUI and `True` during the mass test series. A typical example is shown here:
``` python
if not silent:
    print(f"""
Model for circular map created:
    Number of cities: {self.N:3d}
    Fixed city:       {self.fixed_city:3d}
    Fixed time:       {self.fixed_time:3d}
""")
            

```

Mass output should not be sent to the tab via `print` or logging. Such content can be written to external files (e.g. .html, .png). The relative url (i.e. the filename relative to the current directory) can be used to create a link in the **Setup scenario** with `self.Optimizer.set_load_details_reference`:
``` python
from tabulate import tabulate
...

with open("./O_02_distance_table.html", 'w') as detail:
    detail.write(tabulate(self.distance_table, range(N), showindex="always", tablefmt="html"))

self.Optimizer.set_load_details_reference(url = "O_02_distance_table.html")

```
In the next box the `load` method is added to our `OptimizerModel`. Now the **Setup scenario** GUI can be called.

In [ ]:
from tabulate import tabulate

class Demo_UIFunctions_Model_2(Demo_UIFunctions_Model_1):
    def __init__(self, persistent_file="O_02_Optimizer_UI_functions.dao", **kwargs):
        Demo_UIFunctions_Model_1.__init__(
            self,
            persistent_file=persistent_file,
            **kwargs)
        
    ##################################################################################################################
    # load method (defines content of tab Setup scenario)
    ##################################################################################################################
    def load(self, N=16, fixed_city=0, fixed_time=0, silent=False):
        
        # store paramters in model attributes
        self.N = N
        self.fixed_city = min(fixed_city, N - 1)
        self.fixed_time = min(fixed_time, N - 1)
        
        # calculate distance table
        self.cities = [[100*math.sin(c*2*math.pi/N), 100*math.cos(c*2*math.pi/N)] for c in range(N)]
        
        self.distance_table = [[math.sqrt((self.cities[c_1][0] - self.cities[c_0][0])**2 + (self.cities[c_1][1] - self.cities[c_0][1])**2)
                                 for c_0 in range(N)] for c_1 in range(N)]
        
        # report actions to tab
        self.LOGGER.info(f'Model for circular map with {self.N} cities created.')
        
        if not silent:
            print(f"""
Model for circular map created:
    Number of cities: {self.N:3d}
    Fixed city:       {self.fixed_city:3d}
    Fixed time:       {self.fixed_time:3d}
""")
        # rich output to linked file
        with open("./O_02_distance_table.html", 'w') as detail:
            detail.write(tabulate(self.distance_table, range(N), showindex="always", tablefmt="html"))
        self.Optimizer.set_load_details_reference(url = "O_02_distance_table.html")
    
my_optimizer_2 = Optimizer(Demo_UIFunctions_Model_2(), log_level=logging.INFO) 

### Implementation of mathematical model: `build_qubo`:
The `build_qubo` method has the task to create a qubo for the given business model. In our case the business model is given by the number of cities `self.N`, a fixed city and time step `self.fixed_city` and `self.fixed_time` and the distance table `self.distances`.

#### Layout of the bit array
We want to address the bits with a two dimensional index as $x_{t,c}$ for time steps $t=0,1,...N-1$ and cities $c=0,1,...N-1$. This can be defined as `BitArrayShape` variable:
``` python
BitArrayShape('x', (self.N, self.N))
```
A bit $x_{t,c}$ is assumed to be 1 iff we visit at time $t$ the city $c$. Not all movements are possible, we want to visit in a certain time step $t_F$ the city $c_F$. That means the bit $x_{t_F,c_F}$ is 1 and all bits  $x_{t,c_F}$ with $t \neq t_F$ and $x_{t_F,c}$ with $c \neq c_F$ are 0. To define these constant bit positions we need a numpy array of same square shape with appropriate 0 and 1 entries at the fixed positions and -1 in all other positions of variable bits:
``` python
constant_bits = np.full((self.N, self.N), -1, np.int8)
constant_bits[self.fixed_time, :] = 0
constant_bits[:, self.fixed_city] = 0
constant_bits[self.fixed_time, self.fixed_city] = +1                

```
This can be added to the `BitArrayShape` constructor together with axis names for later visualization. $X=(x_{t,c})$ is the only variable we need for the TSP, so we can define the `VarShapeSet` for the QUBO as:
``` python
self.qubo_vs = VarShapeSet(BitArrayShape('x', (self.N, self.N), constant_bits=constant_bits, axis_names=['Time', 'City'])) 
```
For more possibilities in layout declaration for the bit vector of a QUBO please see the chapter **Structured variables** in the **dadk Introduction**.

#### The polynomials
Now we can define our Hamiltonian `HQ`. We start with
$$
P_{time} = f_{rules} * \sum_{t=0}^{N - 1} \left(1 - \sum_{c=0}^{N - 1} x_{t,c} \right)^2
$$
which can be constructed in Python by:
``` python
self.P_time = BinPol(self.qubo_vs)   
for t in range(self.N):
    self.P_time += (1 - BinPol.sum(Term(1, (('x', t, c),), self.qubo_vs) for c in range(self.N))) ** 2
self.P_time *= factor_rules
```

We can also use `Term` objects and operators to implement such sums. This is demonstrated for 
$$
P_{city} = f_{rules} * \sum_{c=0}^{N - 1} \left(1 - \sum_{t=0}^{N - 1} x_{t,c} \right)^2
$$
by a single statement
``` python
self.P_city = factor_rules *  \
              BinPol.sum(
                (1 - BinPol.sum(Term(1, (('x', t, c),), self.qubo_vs) for t in range(self.N))) ** 2 
                for  c in range(self.N)
              )
```
The minimization target,
$$
P_{distance} = f_{distance} *  \sum_{t=0}^{N - 1} \sum_{c_0=0 \\c_1=0 \\c_0 \ne c_1}^{N - 1} \Delta_{c_0, c_1} x_{t,c_0} x_{(t+1) \% N, c_1} 
$$
which is the effective total length of the traveling route, is here also implemented in one statement:
``` python
self.P_distance = factor_distance * \
                  BinPol.sum(Term(self.distance_table[c_0][c_1], (('x', t, c_0), ('x', (t + 1) % self.N, c_1)), self.qubo_vs)
                             for c_0 in range(self.N) for c_1 in range(self.N) if c_0 != c_1 for t in range(self.N))
```
For more practice in polynomial construction please see the **Polynomials** chapter in the **dadk Introduction**.

The **Optimizer** framework expects the QUBO as attribute `HQ` of the model, so we sum up the polynomials we created so far:
``` python
self.HQ = self.P_time + self.P_city + self.P_distance
```

Now we have constructed our QUBO and can use the **Solve annealing** tab to set the annealing parameters and send it to the solver. 

#### Calculated anneaing parameter
It is also possible to predefine some of the solver parameters e.g. since it needs adjustment with respect to scaling or norming of the QBUO. In this example we calculate a start temperature for annealing by sampling 20 energy values of `HQ` and use half of the estimated standard deviation as start temperature. This value can then be transferred to the annealing parameters:
``` python
recommended_temperature_start = round(0.1 * np.std([self.HQ.compute([randint(0, 1) for _ in range(self.N**2)]) for case in range(20)]), 3)
self.Optimizer.set_calculated_solver_parameter(temperature_start=recommended_temperature_start)
```


In [ ]:
class Demo_UIFunctions_Model_3(Demo_UIFunctions_Model_2):
    ##################################################################################################################
    # build_qubo method (defines content of tab Build QUBO)
    ##################################################################################################################
    def build_qubo(self, factor_distance, factor_rules, silent=False):
        
        # variable bit as default are initialized with -1
        constant_bits = np.full((self.N, self.N), -1, np.int8)
        # all bits for fixed_time are set to constant 1
        constant_bits[self.fixed_time, :] = 0
        # all bits for fixed city are set to constant 0
        constant_bits[:, self.fixed_city] = 0
        # bit for fixed_time in fixed_city is set to constant 1
        constant_bits[self.fixed_time, self.fixed_city] = +1                
        # square array x with constant bits is created as VarShapeSet            
        self.qubo_vs = VarShapeSet(BitArrayShape('x', (self.N, self.N), constant_bits=constant_bits, axis_names=['Time', 'City']))        
                        
        #-P_time-------------------------------------------------------------------------------
        
        self.P_time = BinPol(self.qubo_vs)   
        for t in range(self.N):
            self.P_time += (1 - BinPol.sum(Term(1, (('x', t, c),), self.qubo_vs) for c in range(self.N))) ** 2
        self.P_time *= factor_rules
        
        #-P_city-------------------------------------------------------------------------------

        self.P_city = factor_rules *  BinPol.sum(
            (1 - BinPol.sum(Term(1, (('x', t, c),), self.qubo_vs) for t in range(self.N))) ** 2 
            for  c in range(self.N))

        #-P_distance---------------------------------------------------------------------------
                        
        self.P_distance = factor_distance * BinPol.sum(Term(self.distance_table[c_0][c_1], (('x', t, c_0), ('x', (t + 1) % self.N, c_1)), self.qubo_vs)
                                                       for c_0 in range(self.N) for c_1 in range(self.N) if c_0 != c_1 for t in range(self.N))

        #-HQ-----------------------------------------------------------------------------------

        self.HQ = self.P_time + self.P_city + self.P_distance
        
        # calculate and set start temperature as standard deviation of QUBO energy
        recommended_temperature_start = round(0.1 * np.std([self.HQ.compute([random.randint(0, 1) for _ in range(self.N**2)]) for case in range(20)]), 3)
        print(f'Calculated start_temperature: {recommended_temperature_start}')
        
        self.Optimizer.set_calculated_solver_parameter(temperature_start=recommended_temperature_start)
    
my_optimizer_3 = Optimizer(Demo_UIFunctions_Model_3(), log_level=logging.INFO) 

#### The GUI for **Solve annealing**
The GUI for the **Solve annealing** tab presents all possible parameters to control the annealing process (apart the access to the annealing service which is separated in the annealing profile and can be set via `SetAnnealerProfile()`). There is no special GUI definition required. Nevertheless it is possible to customize this as well. Therefor the parameters `default_solver_parameter`, `demo_solver_parameter` and `calculated_solver_parameter` are used.

`default_solver_parameter` defines default values for the **Solve annealing** tab deviating from the system default values. At any time the user can restore these value by pressing the **Default** button. The values are specified as a dictionary with `FujitsuDASolver` parameters as key and the intended values, e.g.:
``` python
default_solver_parameter={'number_iterations':2000,
                          'temperature_end':0.1,
                          'offset_increase_rate':2.0}
```
The possible names can be found in the GUI display as labels in front of the respective field.

`demo_solver_parameter` allow to hide widgets in certain modes. They are specified similar to the tags in the normal GUI parameter definition as list of dictionaries, where the dictionaries only have the keys `name` and `tags`.  If a parameter should occur in all modes there is a convenience feature; instead of a dictionary with `name` and `tags` simply a string with the parameter name may be added to the list like the `number_replicas` in the following example: 
``` python
demo_solver_parameter=[{'name':'number_iterations', 'tags':['demo1', 'demo2']},
                       {'name':'number_runs', 'tags':['demo1']},
                       'number_replicas'],
```

`calculated_solver_parameter` define calculated values for the **Solve annealing** tab. They are displayed but the user **can't** modify these fields any more in the GUI. 
``` python
calculated_solver_parameter={'temperature_start': 50000},
```
The calculated parameters are specified as dictionary of parameter names as keys; the specified value is a default, which can be overwritten with the method `self.Optimizer.set_calculated_solver_parameter(<parameter_name>=<parameter_value>, ...)` e.g. called from he `build_qubo` method. (see at end of the `build_qubo` in the example below).

In [ ]:
class Demo_UIFunctions_Model_4(Demo_UIFunctions_Model_3):
    def __init__(self, persistent_file="O_02_Optimizer_UI_functions.dao", **kwargs):
        Demo_UIFunctions_Model_3.__init__(
            self,
            persistent_file=persistent_file,
            # Define default values for the "Solve annealing" tab.
            # The user can modify these fields, or press the button 
            # "Default" and get these values again.
            default_solver_parameter={'number_iterations':2000,
                                      'temperature_end':0.1,
                                      'offset_increase_rate':2.0
                                     },
            # Define calculated values for the "Solve annealing" tab.
            # The user CAN'T modify these fields any more. The value
            # shown here is a default, which is will be overwritten with 
            # the methode self.Optimizer.set_calculated_solver_parameter(...)
            # see end of build_qubo for an example.
            calculated_solver_parameter={'temperature_start': 50000},
            demo_solver_parameter=[{'name':'number_iterations', 'tags':['demo1', 'demo2']},
                                   {'name':'number_runs', 'tags':['demo1']},
                                   'number_replicas'
                                  ],
            **kwargs)

my_optimizer_4 = Optimizer(Demo_UIFunctions_Model_4(), log_level=logging.INFO)

### Evaluate solution: `prep_result`:
Once the annealing is completed the solution comes back as a bit vector, i.e. the state in the search space with the lowest energy. Since you can do multiple independent runs you can get multiple solutions. The `prep_result` method can receive all those results for sampling purpose; this is achieved by setting the annealing parameter `solution_mode` as `COMPLETE` and declaring a parameter `solution_list`. In the case of optimization `solution_mode` as `QUICK` a parameter `configuration` is easiest way to get the results. The duty of the `prep_result` method is the conversion of bits to a meaningful information about the problem. In our case the best tour for the traveling salesmen is calculated from the bit values.

In this example we save the configuration as attribute of the model object. Further processing is only done in the non silent mode after an explicit call of the method from GUI. First we calculate and print the energy of the QUBO and of the polynomials it consists of:
``` python
e_HQ = self.HQ.compute(self.configuration)
print("HQ         = %10.2F" % e_HQ)
```

We can check the values for $P_{time}$ and $P_{city}$, which must be zero for an admissible solution.

For a valid solution we examine the bit vector of the result. From the 'solution' we can then extact one bit array, in our case that with the name `x`. This is now a 2-dimensonal array with axis for time and city.
``` python
self.start_table = solution_list.min_solution.extract_bit_array('x')
self.route = [np.argmax(self.start_table.data[t,:]) for t in range(self.N)]
```

Finally the route array is used to print the steps, their distance and the cumulated travel distance.

In [ ]:
class Demo_UIFunctions_Model_5(Demo_UIFunctions_Model_4):
    ##################################################################################################################
    # prep_result method (evaluates result of annealing defines content of tab Solve annealing)
    ##################################################################################################################
    def prep_result(self, solution_list:SolutionList, silent=False):

        configuration = solution_list.min_solution.configuration

        self.configuration = configuration # <-- best solution
        self.start_table = None
        self.route = None
              
        if not silent:
            
            e_HQ = self.HQ.compute(self.configuration)
            e_time = self.P_time.compute(self.configuration)
            e_city = self.P_city.compute(self.configuration)
            e_distance = self.P_distance.compute(self.configuration)
            
            print("HQ         = %10.2F" % e_HQ)
            print("P_time     = %10.2F" % e_time)
            print("P_city     = %10.2F" % e_city)
            print("P_distance = %10.2F" % e_distance)
            print("")
            if e_time == 0 and e_city == 0:

                self.start_table = solution_list.min_solution.extract_bit_array('x')
                self.route = [np.argmax(self.start_table.data[t,:]) for t in range(self.N)]

                distances = [self.distance_table[self.route[t]][self.route[(t + 1) % self.N]] for t in range(self.N)]
                self.output_table = tabulate(zip(range(self.N),
                                                 self.route,
                                                 self.route[1:] + self.route[:1],
                                                 distances,
                                                 np.cumsum(distances)
                                                ),
                                             ['t', 'from', 'to', 'distance', 'total'], tablefmt="html")
                display(HTML(self.output_table))
            else:
                print("invalid solution, try again!")
    
my_optimizer_5 = Optimizer(Demo_UIFunctions_Model_5())  

The call button in the **Solve annealing** tab sends the QUBO to an annealer, waits for the results and calls `prep_result` to transform the result and to display some information in the GUI. More information about the annealing process is available in the **Anneal tracker** tab. On the top of the tab is a toolbar to select between the most recent annealing results. After selection a bar chart with time consumption is displayed. The upper line shows the main actions from client perspective and the respective time consumption. The following lines split some of the actions into sub actions on the next level. This gives an overview and helps to find the critical parts for end to end performance. 

The toolbar allows to select and display the parameter values of all steps and additional statistical information about the annealing process.

If the annealing was executed on an emulation and the annealing parameter `graphics` was set to `Single` or `All` then graphical output about the annealing process is display as well. There are 6 charts to understand better the search process.

After a simulated annealing there are 6 charts displaying indicators of the process. 
![Temperature and waiting cycles](images/temperature.jpg "Fig.1: Temperature and waiting cycles")
<center>Fig.1: Temperature and waiting cycles</center>
The red line in Fig. 1 shows the temperature in logarithmic scale and helps for orientation. On the x axis we have the iterations. The runs are displayed sequentially, the first 2000 steps show the first run, the steps between 2000 and 4000 the second run. The blue boxes show waiting cycles. The annealer tries to find better places in the search space in small steps of one bit flip (Hamming distance 1). We try all bits in each step. If we find a lower energy for a bit flip, then we have a candidate. If the energy is increased for a new configuration, then a random experiment decides if this is a candidate or not. The higher the energy increase and the lower the temperature the lower is the probability to decide for candidate. Afterwards one of the candidates is selected randomly. Waiting cycle means, that we did not find any candidate. If waiting cycles get high pretty early, then running in higher temperature can be a counter measure. The triangle marks the position, where the minimal energy was reached first. If this occurs very early at high temperature, the reason could be an unstable system, so lower final temperature could lead to stable processes and find a better minimum.

![Energy](images/energy.jpg "Fig.2: Energy track")
<center>Fig.2: Energy track</center>
Fig. 2 shows the energy decrease. The gray curve is the total energy in logarithmic scale. In the beginning for most QUBOs there is a short and fast decline expected. In this phase all the violations caused by a random starting point are healed and we always have good candidate steps. Here that takes the first hundred steps. Then we see the energy curve going up and down. Due to thermal noise we have energy increases and then again greedy steps going to lower energy. When we are stuck in a local minimum and don't escape by thermal noise (a waiting cycle), then we try in the next step with an energy offset, which is increased in every subsequent waiting cycle. This is shown by the blue and red curve. Blue is the valley energy of the system when the dynamic offset is first used. After some cycles we will get candidates again, we have reached a hill and that energy is written in the red curve. Here we see most often a difference of 50 between valley and next hill. Since we increase the energy by 2 in waiting cycle, that fits to the observation of 25 waiting cycles. We could next try a higher dynamic offset to reduce the waiting cycles and accelerate the convergence.

![Valley Hill](images/valley_hill_energy.jpg "Fig.3: Valley and hill energy")
<center>Fig.3: Valley and hill energy</center>
The graphic shows only the valley and the hill energy bound to the left and right axis. Sometimes this can help to see exact values easier. 

![Bit flips](images/bit_flips.jpg "Fig.4: Bit flips")
<center>Fig.4: Bit flips</center>
Fig. 4 shows the bit flips over time. On the x axis again we have the iterations. The runs are displayed sequentially, the first 2000 steps show the first run, the steps between 2000 and 4000 the second run. On the y axis the bits are denoted. Each horizontal line show the changes of a single bit, blue dot indicates flip to 0 while red dot means flip to 1. We can clearly see how the probability of a flip deceases over time when the temperature goes down

![Heatmap](images/bit_flips_heatmap.jpg "Fig.5: Bit flips heatmap")
<center>Fig.5: Bit flips heatmap</center>

In the visualization in Fig. 5 the bit flip activity is visualized as heatmap. Here we see some activity in the beginning for all bits. That is again the fast optimizing start, where all violations are healed. Here we see also activity for bit 40. But it is not very significant. The relative uniform behavior can be expected since all bits on a circle should have similar behavior when switched on and off. For uneven distributed cities this might change and flipping back and forth between cities very close to each other without any progress might mask other bits, which possibly would give better results. After a deeper analysis we might find some readjustment of the QUBO. In this case we are doing traveling salesman and it turned out that the active bits belong to exposed cities with far distances to their next neighbors. Switching their connections has a big effect and is therefore often tried back and forth. One idea can be a transformation to a logically equivalent map with less exposed cities. 

![Bit flips histogram](images/bit_flips_histogram.jpg "Fig.6: Bit flips histogram")
<center>Fig.6: Bit flips histogram</center>
The last chart shows a histogram of flip counts of each particular bit.

### Summarize result: `report`:
In the `report` method the results can be presented in more detail. In this example we use the visualization feature of `BitArray` objects. Since we declared axis names in the `BitArrayShape` we get a square visualization of the bits including axis label with one statement.

In [ ]:
class Demo_UIFunctions_Model_6(Demo_UIFunctions_Model_5):
        
    ##################################################################################################################
    # report method (defines content of Report tab)
    ##################################################################################################################
    def report(self, silent=False):
        
        if self.start_table is not None:
            self.start_table.draw(figsize=(6.0, 6.0))
    
my_optimizer_6 = Optimizer(Demo_UIFunctions_Model_6(), immediate_load=True, log_level=logging.INFO) 

It is possible to change the titles of the report tabs by adding adding them to the constructor. Beside **report** there are 9 additional report tabs possible. The corresponding methods are named **report1** to **report9**, for a total of up to 10 report tabs.

In [ ]:
class Demo_UIFunctions_Model_7(Demo_UIFunctions_Model_6):
    def __init__(self, persistent_file="O_02_Optimizer_UI_functions.dao", **kwargs):
        Demo_UIFunctions_Model_6.__init__(
            self,
            persistent_file=persistent_file,
            # new add parameter, to define titels of different report tabs
            report_title_list=["Bit map report",      "Route report"],
            #                  for method "report"    for method "report1"
            **kwargs)
        
    ##################################################################################################################
    # report1 method (defines content of additional "Report" tab)
    ##################################################################################################################
    def report1(self, silent=False):
        print('This is the best found roundtrip:')
        if self.start_table is not None:
            display(HTML(self.output_table))
    
my_optimizer_7 = Optimizer(Demo_UIFunctions_Model_7(), immediate_load=True, log_level=logging.INFO) 

### Visualization: `draw`:
The `draw` method has the same functionality as the `report` method. It provides the content for the tab **Visualization** and should be used for a graphical representation of the problem and/or result. If you want an individual title for the tab just use the attribute **visualization_title** in the constructor.

In the following example the cities and the selected round trip as a map in the 2-dimensional plane are plotted by `matplotlib`and `networkx`.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

class Demo_UIFunctions_Model_8(Demo_UIFunctions_Model_7):
    def __init__(self, persistent_file="O_02_Optimizer_UI_functions.dao", **kwargs):
        Demo_UIFunctions_Model_7.__init__(
            self,
            persistent_file=persistent_file,
            # modify the title for the Visualization tab to "Plot"
            visualization_title="Plot", # Default is "Visualization"
            **kwargs)

    ##################################################################################################################
    # draw method (defines content of optional "Visualization" tab)
    ##################################################################################################################
    def draw(self):

        if self.start_table is not None and self.route is not None:
            self.graph = nx.Graph()

            nodes={}
            for i in range(self.N):
                nodes[i] = str(i);
                self.graph.add_node(nodes[i], pos=(self.cities[i][0], self.cities[i][1]))

            path = [[self.route[t], self.route[(t + 1) % self.N]] for t in range(self.N)]

            for i in range(self.N):
                for j in range(i+1, self.N):
                    if [i, j] in path or [j, i] in path: 
                        self.graph.add_edge(nodes[min(i,j)], nodes[max(i,j)], weight=10)
                    else:
                        self.graph.add_edge(nodes[min(i,j)], nodes[max(i,j)], weight=1)

            fig = plt.figure(num='Graph (created at %s)' % datetime.now(), figsize=(8.0, 8.0))

            pos=nx.get_node_attributes(self.graph,'pos')

            # nodes
            nx.draw_networkx_nodes(self.graph, pos, nodelist=self.graph.nodes, node_size=100, node_color="red")

            # edges        
            edges_bold = [(u, v) for (u, v, d) in self.graph.edges(data=True) if d['weight'] == 10]
            nx.draw_networkx_edges(self.graph, pos, edgelist=edges_bold, width=3, edge_color="black")

            edges_thin = [(u, v) for (u, v, d) in self.graph.edges(data=True) if d['weight'] == 1]
            nx.draw_networkx_edges(self.graph, pos, edgelist=edges_thin, width=1, alpha=0.1, edge_color="black")

            # labels
            nx.draw_networkx_labels(self.graph, pos, font_size=6, font_family='sans-serif')

            plt.axis('off')
            plt.show()
    
my_optimizer_8 = Optimizer(Demo_UIFunctions_Model_8(), immediate_load=True, log_level=logging.INFO) 

### Test your model: `pareto`
The **Optimizer** framework supports the recording of test series and displays the results in the **Pareto** tab. For the recording the complete process including `load`, `build_qubo` and `prep_result` method is executed multiple times. At the end of each process the `pareto` method is called to select the plotting data. The method returns a dictionary with keys as axis names and values as numbers for plotting. Typically these are numerical values, since they should be plotted. Categorial values like strings or tuples are plotted with the sequence number of the test case. Each call generates a data set with data for the respective test. The returned dictionaries must have the same structure in all calls.

``` python
def pareto(self):
    return {
        'HQ' : self.HQ.compute(self.configuration),
        'P_time': self.P_time.compute(self.configuration),
        'P_city': self.P_city.compute(self.configuration),
        'P_distance': self.P_distance.compute(self.configuration)
    }
```

#### The **Pareto curve** test series
No direct GUI specifications are required for the **Pareto curve** tab. Instead, the data generation for the test series is defined here. The tab displays a series of complete solutions generated by going through `load`, `build_qubo` and solving including `prep_results` and `pareto` for data selection. The parameter setting for this test series is specified as a dictionary with the three entries `var_object`, `var_attribute` and `var_values`:

``` python
#                            modulated parameter 1     modulated parameter 2
pareto_setting={
    'var_object':         [ 'build_qubo_parameter',   'build_qubo_parameter'],
    'var_attribute':      [ 'factor_rules',           'factor_distance'],
    'var_values':         [ [50, 75, 100, 125, 150],  [1.75, 1.50, 1.00, 0.75, 0.50] ],
    'cartesian_product':  False
}
```
It is expected that each of these three lists is of identical length, one entry per parameter to be modulated in the test series. The entries in `var_object` name the method behind a tab that receives the parameters and can be one of the values `load_parameter`, `build_qubo_parameter` and `solve_parameter`. The entries of `var_attribute` name the respective parameters of these methods. The `var_values` contain lists of equal length, i.e. the number of test cases; for each test case these lists contain the value to be applied to the respective parameter. The test driver modulates exactly the specified parameters, while all other parameters are taken from the current definition in the GUI (more precisely, the values specified in the last call to `load`, `build_qubo` and the solution are applied). If a certain fixed value is assumed for a certain parameter (independent of the user settings in the GUI), then it is recommended to specify this value in the `pareto_setting` with a list of identical values as well.

For bigger test series it is possible, to generate all value combinations of the specified parameters. This is controlled by setting the entry ```'cartesian_product'``` to ```True```. **Caution**: The number of tests is the product of the length of the lists, so that a large number of tests can be specified with a few parameters. 

##### Pareto GUI 

The Pareto GUI supports graphical evaluation of the recorded test series. All parameter values and recorded values can be attached to the x-axis or the left or right y-axis.  Value sequences attached to the right y-axis can be transformed by calculation before displayed. It is possible to visualize multiple value sequences simultaneously e.g. for comparison. For the pareto plot, it is possible to adapt the precision of x- and y-axis. In particular, the number of decimal places can be chosen for each of the axes.
In the picture below, no decimal place is chosen for the x-axis. Similarly, two digits precision or the exponential format are chosen for the left and right y-axis, in order to present the values of HQ with an appropriate accuracy. 

![pareto_tsp.png](images/pareto_tsp.png)

This way, it is possible to adapt the layout to the type of data to be plotted. On the control tab it is possible to tag an evaluation setup; this creates a button below the plot and allows an easy switching between different visualization. The tags are stored in the dao file and are there available in subsequent sessions.

![pareto_tsp.png](images/pareto_tagging.png)

#### The **Timeseries** test
The **Timeseries** allows to run test cases of different sizes to see the scaling behavior of the whole optimization. These tests are sorted by the number of bits for the generated qubo and several measured time consumptions can be visualized in the register **Timeseries**. Similar to the Pareto tests, the tests are specified by value sets for certain parameters. 
``` python
timeseries_setting={
    'var_object':    ['load_parameter'],
    'var_attribute': ['N'],
    'var_values':    [[ 4, 8, 12, 16, 20, 24]],
    'max_bits': 8192        
},
```
In this case, however, the complete Cartesian product of the value lists is generated as test scenarios. This means that the lists can have different lengths. **Caution**: The number of tests is the product of the length of the lists, so that a large number of tests can be specified with a few parameters. The key `max_bits` limits the fully executed tests, but only the annealing step is omitted. The `load` and the `build_qubo` methods are executed anyway, so the test may take a considerable amount of time.

The following code defines a `pareto` method and the data structures for Pareto and time series in a reversed `__init__` method. You can use the controls in the tabs to change the diagrams or to regenerate the complete test series if the input parameters from the previous tabs have been changed.

In [ ]:
class Demo_UIFunctions_Model_9(Demo_UIFunctions_Model_8):
    def __init__(self, persistent_file="O_02_Optimizer_UI_functions.dao", **kwargs):
        Demo_UIFunctions_Model_8.__init__(
            self,
            persistent_file=persistent_file,
            # input data for pareto test series
            pareto_setting={
                'var_object':['build_qubo_parameter', 'build_qubo_parameter'],
                'var_attribute':['factor_rules', 'factor_distance'],
                'var_values':[
                    [50, 75, 100, 125, 150],
                    [1.75, 1.50, 1.00, 0.75, 0.50]
                ]
            },
            # input data for performance test series
            timeseries_setting={
                'var_object':['load_parameter'],
                'var_attribute':['N'],
                'var_values':[[ 4, 8, 12]],#, 16, 20, 24]],
                'max_bits': 8192        
            },
            **kwargs)
        
    ##################################################################################################################
    # pareto method (defines records for pareto curves)
    ##################################################################################################################
    def pareto(self):

        return {
            'HQ' : self.HQ.compute(self.configuration),
            'P_time': self.P_time.compute(self.configuration),
            'P_city': self.P_city.compute(self.configuration),
            'P_distance': self.P_distance.compute(self.configuration)
        }
            
my_optimizer_9 = Optimizer(Demo_UIFunctions_Model_9(), auto_load=True, log_level=logging.INFO)

#### Generation of Pareto and Time Series via immediate_run and handling dao Files

Note that a test for pareto or time series can only be run after the Call button in each of the tabs `Setup scenario`, `Build QUBO` and `Solve Annealing` has been executed at least once.
For each test in a pareto or time series setting, a new "child" Optimizer object is initialized with a copy of the parameter settings of the current OptimizerModel, except that the respective parameters in the pareto or time series definition are replaced by those of the current test. Each of these child Optimizer objects is run in immediate_run mode, and the produced dao files are stored in a folder which has the name of the base OptimizerModel object's dao file. Assuming a notebook `notebook.ipynb` and dao file `base.dao` in a working directory the structure is the following:

```
   ...
   +---_ working_directory
   |   +---_base
   |   |   +---_pareto
   |   |   |   +--- 0.dao
   |   |   |   +--- 1.dao
   |   |   |   +--- ...
   |   |   +---_pareto_running
   |   |   |   +--- 0.dao
   |   |   |   +--- 1.dao
   |   |   |   +--- ...
   |   |   +---_timeseries
   |   |   |   +--- 0.dao
   |   |   |   +--- 1.dao
   |   |   |   +--- ...
   |   |   +---_timeseries_running
   |   |       +--- 0.dao
   |   |       +--- 1.dao
   |   |       +--- ...
   |   +--- notebook.ipynb
   |   +--- base.dao
   |   |   ...
   
```

A directory with the same name as the dao file contain a subdirectory `pareto` with all output of a complete test series. For every test case it contains a dao file name by the test case index `0.dao`, `1.dao`, ... Depending `OPTIMIZER_SETTINGS` the directory might contain additional files per test case (e.g. `OPTIMIZER_SETTINGS.Compose.create_summary_excel=True`). While a pareto test series is running, there is a second sub folder `pareto_running`, where the test specific files are stored. Once the series is complete the `pareto_running` folder replaces the `pareto` folder and a result summary is included into the base dao file. Therefore a previous test series result is available until a new test series is complete. If a test series is interrupted or fails, then an incomplete `pareto_running` folder exists in the base directory. This is recognized when starting the optimizer GUI again; in this case a button for recovery is offered, which continues the pareto test series instead of regenerating a complete new series. 


Per default, the folders with pareto or time series dao files are not deleted after the respective test has been finished. In order to delete such a folder and its content, you have the possibility to press a button below the graphic in the `Pareto curve` or `Timeseries` tab.

If you do not wish to store any dao files of a pareto test, you can activate the automatic deletion by setting
```python
OPTIMIZER_SETTINGS.pareto.delete_files = True
```
before you initialize your Optimizer object.
Note that in this case, no delete button and no summary of the test will be shown after the pareto test.
Similarly, you can activate automatic deletion of test series dao files by setting
```python
OPTIMIZER_SETTINGS.timeseries.delete_files = True
```
before you initialize your Optimizer object.
In this case, you will find no delete button in the `Timeseries` tab after the test is finished.

For the time series files a similar process is implemented. The main difference is that there is no recovery for the time series test.

## At a glance: The complete implementation in a box
Normally an `OptimizerModel` is written as one piece of code. It can be managed as an external file edited by a Python development environment, which is recommended for bigger projects. Or the class definition is written in one box as in the example below. You can use the following box to define the model again. This is prerequisite for the following chapters.

In [ ]:
from dadk.Optimizer import *

from tabulate import tabulate
import networkx as nx
import matplotlib.pyplot as plt

class Demo_UIFunctions_Model(OptimizerModel):
    ##################################################################################################################
    # constructor (defines GUI and test declaration)
    ##################################################################################################################
    def __init__(self, persistent_file="O_02_Optimizer_UI_functions.dao"):
        OptimizerModel.__init__(
            self,
            name = 'Demo_UIFunctions',
            load_parameter=[
                {'name':'N', 'type':'int_slider', 'value':12, 'min':2, 'max': 90, 
                 'label':'Size', 'description':'problem size', 'tags':['demo1', 'demo2']},
                {'name':'fixed_city', 'type':'int_slider', 'value':0, 'min':0, 'max':90, 
                 'label':'fixed city', 'description': '', 'tags':['demo1']},
                {'name':'fixed_time', 'type':'int_slider', 'value':0, 'min':0, 'max':90, 
                 'label':'fixed time', 'description':'', 'tags':['demo1']}
            ],            
            build_qubo_parameter=[
                {'name':'factor_rules', 'type':'float_slider', 'value':100.0, 'min':0.0, 'max':10**6,
                 'label':'factor rules'},
                {'name':'factor_distance', 'type':'float_slider', 'value':1.0, 'min':0.0, 'max':10**3,
                 'label':'factor distance'}
            ], 
            #
            # Define default values for the "Solve annealing" tab.
            # The user can modify these fields, or press the button 
            # "Default" and get these values again.
            #
            default_solver_parameter={'number_iterations':2000,
                                      'temperature_end':0.1,
                                      'offset_increase_rate':2.0
                                     },
            #
            # Define calculated values for the "Solve annealing" tab.
            # The user CAN'T modify these fields any more. The value
            # shown here is a default, which is will be overwritten with 
            # the methode self.Optimizer.set_calculated_solver_parameter(...)
            # see end of build_qubo for an example.
            #
            calculated_solver_parameter={'temperature_start': 50000},
            demo_solver_parameter=[{'name':'number_iterations', 'tags':['demo1', 'demo2']},
                                   {'name':'number_runs', 'tags':['demo1']},
                                   'number_replicas'
                                  ],
            
            # new add parameter, to define titels of different report tabs
            report_title_list=["Bit map report",      "Route report"],
            
            # modify the title for the Visualization tab to "Plot"
            visualization_title="Plot", # Default is "Visualization"
            
            # input data for pareto test series
            pareto_setting={
                'var_object':['build_qubo_parameter', 'build_qubo_parameter'],
                'var_attribute':['factor_rules', 'factor_distance'],
                'var_values':[
                    [50, 75, 100, 125, 150],
                    [1.75, 1.50, 1.00, 0.75, 0.50]
                ]
            },
            # input data for performance test series
            timeseries_setting={
                'var_object':['load_parameter'],
                'var_attribute':['N'],
                'var_values':[[ 4, 8, 12]],#, 16, 20, 24]],
                'max_bits': 8192        
            },
            persistent_file=persistent_file
        )
    
    ##################################################################################################################
    # load method (defines content of tab Setup scenario)
    ##################################################################################################################
    def load(self, N=16, fixed_city=0, fixed_time=0, silent=False):
        
        # store paramters in model attributes
        self.N = N
        self.fixed_city = min(fixed_city, N - 1)
        self.fixed_time = min(fixed_time, N - 1)
        
        # calculate distance table
        self.cities = [[100*math.sin(c*2*math.pi/N), 100*math.cos(c*2*math.pi/N)] for c in range(N)]
        
        self.distance_table = [[math.sqrt((self.cities[c_1][0] - self.cities[c_0][0])**2 + (self.cities[c_1][1] - self.cities[c_0][1])**2)
                                 for c_0 in range(N)] for c_1 in range(N)]
        
        # report actions to tab
        self.LOGGER.info(f'Model for circular map with {self.N} cities created.')
        
        if not silent:
            print(f"""
Model for circular map created:
    Number of cities: {self.N:3d}
    Fixed city:       {self.fixed_city:3d}
    Fixed time:       {self.fixed_time:3d}
""")
        # rich output to linked file
        with open("./O_02_distance_table.html", 'w') as detail:
            detail.write(tabulate(self.distance_table, range(N), showindex="always", tablefmt="html"))
        self.Optimizer.set_load_details_reference(url = "O_02_distance_table.html")
        
    ##################################################################################################################
    # build_qubo method (defines content of tab Build QUBO)
    ##################################################################################################################
    def build_qubo(self, factor_distance, factor_rules, silent=False):
        
        # variable bit as default are initialized with -1
        constant_bits = np.full((self.N, self.N), -1, np.int8)
        # all bits for fixed_time are set to constant 1
        constant_bits[self.fixed_time, :] = 0
        # all bits for fixed city are set to constant 0
        constant_bits[:, self.fixed_city] = 0
        # bit for fixed_time in fixed_city is set to constant 1
        constant_bits[self.fixed_time, self.fixed_city] = +1                
        # square array x with constant bits is created as VarShapeSet            
        self.qubo_vs = VarShapeSet(BitArrayShape('x', (self.N, self.N), constant_bits=constant_bits, axis_names=['Time', 'City']))        
                        
        #-P_time-------------------------------------------------------------------------------
        
        self.P_time = BinPol(self.qubo_vs)   
        for t in range(self.N):
            self.P_time += (1 - BinPol.sum(Term(1, (('x', t, c),), self.qubo_vs) for c in range(self.N))) ** 2
        self.P_time *= factor_rules
        
        #-P_city-------------------------------------------------------------------------------

        self.P_city = factor_rules *  BinPol.sum(
            (1 - BinPol.sum(Term(1, (('x', t, c),), self.qubo_vs) for t in range(self.N))) ** 2 
            for  c in range(self.N))

        #-P_distance---------------------------------------------------------------------------
                        
        self.P_distance = factor_distance * BinPol.sum(Term(self.distance_table[c_0][c_1], (('x', t, c_0), ('x', (t + 1) % self.N, c_1)), self.qubo_vs)
                                                       for c_0 in range(self.N) for c_1 in range(self.N) if c_0 != c_1 for t in range(self.N))

        #-HQ-----------------------------------------------------------------------------------

        self.HQ = self.P_time + self.P_city + self.P_distance
        
        # calculate and set start temperature as standard deviation of QUBO energy
        temp_qubo = self.HQ.make_qubo()
        recommended_temperature_start = round(0.1 * np.std([self.HQ.compute([random.randint(0, 1) for _ in range(self.N**2)]) for case in range(20)]), 3)
        print(f'Calculated start_temperature: {recommended_temperature_start}')
        
        self.Optimizer.set_calculated_solver_parameter(temperature_start=recommended_temperature_start)
    
    ##################################################################################################################
    # prep_result method (evaluates result of annealing defines content of tab Solve annealing)
    ##################################################################################################################
    def prep_result(self, solution_list:SolutionList, silent=False):

        configuration = solution_list.min_solution.configuration

        self.configuration = configuration # <-- best solution
        self.start_table = None
        self.route = None
              
        if not silent:
            
            e_HQ = self.HQ.compute(self.configuration)
            e_time = self.P_time.compute(self.configuration)
            e_city = self.P_city.compute(self.configuration)
            e_distance = self.P_distance.compute(self.configuration)
            
            print("HQ         = %10.2F" % e_HQ)
            print("P_time     = %10.2F" % e_time)
            print("P_city     = %10.2F" % e_city)
            print("P_distance = %10.2F" % e_distance)
            print("")
            if e_time == 0 and e_city == 0:

                self.start_table = solution_list.min_solution.extract_bit_array('x')
                self.route = [np.argmax(self.start_table.data[t,:]) for t in range(self.N)]

                distances = [self.distance_table[self.route[t]][self.route[(t + 1) % self.N]] for t in range(self.N)]
                self.output_table = tabulate(zip(range(self.N),
                                                 self.route,
                                                 self.route[1:] + self.route[:1],
                                                 distances,
                                                 np.cumsum(distances)
                                                ),
                                             ['t', 'from', 'to', 'distance', 'total'], tablefmt="html")
                display(HTML(self.output_table))
            else:
                print("invalid solution, try again!")
           
    ##################################################################################################################
    # report method (defines content of Report tab)
    ##################################################################################################################
    def report(self, silent=False):
        
        if self.start_table is not None:
            self.start_table.draw(figsize=(6.0, 6.0))
    
    ##################################################################################################################
    # report1 method (defines content of additional Report tab)
    ##################################################################################################################
    def report1(self, silent=False):
        print('This is the best found roundtrip:')
        if self.start_table is not None:
            display(HTML(self.output_table))
    
    ##################################################################################################################
    # draw method (defines content of optional "Visualization" tab)
    ##################################################################################################################
    def draw(self):
        if self.start_table is not None and self.route is not None:
            self.graph = nx.Graph()

            nodes={}
            for i in range(self.N):
                nodes[i] = str(i);
                self.graph.add_node(nodes[i], pos=(self.cities[i][0], self.cities[i][1]))

            path = [[self.route[t], self.route[(t + 1) % self.N]] for t in range(self.N)]

            for i in range(self.N):
                for j in range(i+1, self.N):
                    if [i, j] in path or [j, i] in path: 
                        self.graph.add_edge(nodes[min(i,j)], nodes[max(i,j)], weight=10)
                    else:
                        self.graph.add_edge(nodes[min(i,j)], nodes[max(i,j)], weight=1)

            fig = plt.figure(num='Graph (created at %s)' % datetime.now(), figsize=(8.0, 8.0))

            pos=nx.get_node_attributes(self.graph,'pos')

            # nodes
            nx.draw_networkx_nodes(self.graph, pos, nodelist=self.graph.nodes, node_size=100, node_color="red")

            # edges        
            edges_bold = [(u, v) for (u, v, d) in self.graph.edges(data=True) if d['weight'] == 10]
            nx.draw_networkx_edges(self.graph, pos, edgelist=edges_bold, width=3, edge_color="black")

            edges_thin = [(u, v) for (u, v, d) in self.graph.edges(data=True) if d['weight'] == 1]
            nx.draw_networkx_edges(self.graph, pos, edgelist=edges_thin, width=1, alpha=0.1, edge_color="black")

            # labels
            nx.draw_networkx_labels(self.graph, pos, font_size=6, font_family='sans-serif')

            plt.axis('off')
            plt.show()
        
    ##################################################################################################################
    # pareto method (defines records for pareto curves)
    ##################################################################################################################
    def pareto(self):
        return {
            'HQ' : self.HQ.compute(self.configuration),
            'P_time': self.P_time.compute(self.configuration),
            'P_city': self.P_city.compute(self.configuration),
            'P_distance': self.P_distance.compute(self.configuration)
        }

### Instances of  `Demo_UIFunctions_Model(OptimizerModel)`
Instances of the inherited `OptimizerModel` class - in this case instances of `Demo_UIFunctions_Model(OptimizerModel)` - contain all modeling information for a certain optimization process. Typically these are business data objects, QUBOs and optimization results. Parameters controlling these steps in this process are also part of the model instance. A model instance can simply created as follows:    

In [ ]:
my_first_model = Demo_UIFunctions_Model()       

### Starting the GUI
Interaction with a model instance is done via the `Optimizer` GUI. To start the GUI an `Optzimizer` object for our model instance is created.

In [ ]:
my_first_optimizer = Optimizer(my_first_model, auto_load=True, log_level=logging.INFO)

Now the GUI is ready to use. Parameters can be specified, and actions called. It is possible to reload the previous or the default parameters for each step. 

Typically the tabs are used from left to right. First the logical scenario is prepared. Then the mathematical model in form of a QUBO is created. This is afterwards solved by a call to the Digital Annealer. Results can then be inspected in report and visualization tabs. The Pareto tab displays and refreshes a test series for an interactive plot. The Timeseries tab displays the results of a load test. 

Every time a `Call` button is pressed the parameter values are stored persistently. When the Optimizer is instantiated a second time by running the box again, the GUI will automatically load the latest values of the parameters. The methods `load`, `build_qubo` and `prep_result` with the last recorded optimization result are called automatically. This functionality brings you back to the most recent state  after reloading the complete notebook as well.

If you don't want the automatic execution of the methods at start time of the GUI, you can suppress it with the stop button, which is offered for some seconds. You can skip this waiting time and start with the execution immediately using the green start button.

### Variations of the GUI
The look and the behavior of the GUI can be modified by the use of parameters. It is possible to hide certain tabs, get additional tabs, influence the starting behavior and more.

In [ ]:
another_optimizer = Optimizer(
    Demo_UIFunctions_Model(),
    mode='demo1',
    auto_load=False,            # enabbe / disable loading pre dialog,          default is True   
    immediate_load=True,        # load persitent data immediately,              default is False
    immediate_run=False,        # run notebook based on persitent data          default is False
    batch_mode=False,           # if True, no GUI will be created               default is False
                                #    please only use in combination with immediate_run
    show_config=False,          # enable / disable tab 'Select configuration',  default is True
    show_build=True,            # enable / disable tab 'Build QUBO',            default is True
    show_solve=True,            # enable / disable tab 'Solve annealing',       default is True
    show_track=False,           # enable / disable tab 'Anneal tracker',        default is True 
    show_report=False,          # enable / disable tab 'Report',                default is True
    show_visual=False,          # enable / disable tab 'Visualization',         default is True
    show_pareto=False,          # enable / disable tab 'Pareto curve',          default is True
    show_timeseries=False,      # enable / disable tab 'Timeseries',            default is True
    read_only=True,             # write protect the persitent file,             default is False
    log_level=logging.INFO,     # log level for OptimizerModel logging          default is logging.ERROR (=40)
    log_format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',       # format string for log messages
)

### Input and result recording: the `dao` files
We have seen, that all parameters and results of a `OptimizerModel` are automatically stored persistently and reloaded. In our current example the data is stored in the file `O_02_Optimizer_UI_functions.dao`, which is the default value of the constructor parameter `persistent_file` of our `OptimizerModel` class `Demo_UIFunctions_Model`. It is possible to define any other filename for this parameter. This allows to define multiple independent persistent model instances as shown in the next example. 

In [ ]:
another_optimizer = Optimizer(Demo_UIFunctions_Model(persistent_file='O_02_Optimizer_UI_functions_2.dao'))    

### Managing multiple `dao` files:
In many cases the used persistence file is fixed and therefor written in the `Optimizer` call. If you want record man different cases in different `dao` file, use the option
``` python
show_config=True
```
This displays an extra tab **Select configuration**. In the first line the currently used `dao` file is displayed it might be empty in the beginning. With the `Load from ...` you can navigate through the disk for different `dao` files. Once you have selected a file some content information is displayed but the file is not loaded, the `Current` field is not changed. It needs the `call` button to make the selected file the current one and load it. An additional call will not load the content again but persist the description, if it was changed. It is also possible to change the `Save as` field. A subsequent call will copy the current persistent setting to the new file and make that the current file.

In [ ]:
another_optimizer = Optimizer(
    Demo_UIFunctions_Model(persistent_file='O_02_Optimizer_UI_functions.dao'),
    auto_load=False,
    show_config=True
)   

A simple possibility to select from multiple  `dao` files is the `select_instance` method of the `Optimizer` class. It creates a select list to allow switching between model instances of a list of available `dao` file. By default the `dao` files are searches in the current working directory. It is also possible to define another directory relative to the working directory and a regular expression as pattern, to filter the list. Regular expressions are searched anywhere in the filename, if only matches at the begin or end or a complete match with the filename is intended the meta-characters `^` and `$` have to be used at the begin or end of the pattern.

In [ ]:
Optimizer.select_instance(
    optimizer_model=Demo_UIFunctions_Model,         # `OptimizerModel` class to be used with the `dao` files
    dao_path='.',                                   # path to `dao` files
    dao_pattern='^O_02_',                           # regular expression to filter `dao` files
    initial_selection=None,                         # None or the initial selection for preloading a certain `dao` file , here e.g. 'O_02_Optimizer_UI_functions.dao', 
    immediate_load=True, read_only=True             # all paramters of the `Optimizer` constructor can be used to define look and feel
)